In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [9]:
# load link counts data

import json

with open('../temp_files/link_day_minute_Jan_dict_JSON.json', 'r') as json_file:
    link_day_minute_Jan_dict_JSON = json.load(json_file)

In [11]:
week_day_Jan_list = [2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 30, 31]
#week_day_Jan_list = [2, 9, 16, 23, 30]

In [12]:
link_day_minute_Jan_list = []
for link_idx in range(24):
    for day in week_day_Jan_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Jan_list.append(link_day_minute_Jan_dict_JSON[key] ['NT_flow_minute'][minute_idx])

In [13]:
len(link_day_minute_Jan_list)

63360

In [14]:
x = np.matrix(link_day_minute_Jan_list)
x = np.matrix.reshape(x, 24, 2640)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [15]:
np.size(x, 0), np.size(x, 1)

(24, 2640)

In [16]:
x[:,:2]

matrix([[  8995.05938759,   8030.02196823],
        [  8171.52011713,   9699.77080212],
        [ 15959.86338225,  15794.20989479],
        [ 15812.44687184,  14824.26829831],
        [ 15221.81772592,  11841.23156822],
        [  8715.54322512,  10974.2689987 ],
        [  8403.94695721,  10028.26373766],
        [  8574.12371405,  10204.91138194],
        [ 13422.55876947,  16409.93825369],
        [ 15063.11181487,  11500.11635996],
        [ 18108.61398581,  19085.68170638],
        [ 14620.13318487,  15341.42404447],
        [  9762.11861732,   6421.97366671],
        [  6365.70931551,   7619.39827218],
        [  6181.63625046,  11292.25987543],
        [  9315.42242297,   6235.43681936],
        [ 11607.34606222,  10986.8723308 ],
        [ 10573.33969094,  10946.26220292],
        [ 11899.45459204,  12904.79944995],
        [ 11028.5500341 ,  12205.91156769],
        [  8164.74183339,   8603.73202786],
        [  8468.58626816,   8737.32726479],
        [ 11025.33275984,   8864

In [17]:
np.size(A,0), np.size(A,1)

(24, 140)

In [18]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b' * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [7e-02, 1e+01]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.01s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 24
 AA' NZ     : 2.760e+02
 Factor NZ  : 3.000e+02
 Factor Ops : 4.900e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -3.98056856e+06  0.00000000e+00  5.28e+03 3.36e+02  8.97e+05     0s
   1   4.88493567e+05 -2.32787652e+06  8.78e+02 1.61e+02  2.02e+05     0s
   2   4.03765866e+05 -1.36822100e+06  8.78e-04 5.68e-14  3.16e+04     0s
   3  -1.71713576e+05 -4.06068703e+05  8.78e-10 8.53e-14  4.1

In [19]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Jan_weekday_NT.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [20]:
lam_list

[2.3623952705493528e-11,
 15271.92220697726,
 2.614337843237966e-11,
 6.527142463118071e-08,
 3852.3022784473983,
 1.0942729584830326e-07,
 7.3589689229596535e-09,
 4383.180689768528,
 1.62492422550515e-10,
 11518.456550143079,
 1.6208890423937536e-10,
 1.6249242240229085e-10,
 1.622514896872986e-10,
 1.631806560241298e-10,
 17315.88334577116,
 5038.154623272669,
 1.7037078993242902e-07,
 6.52714328294161e-08,
 5474.285073795809,
 17990.74771055783,
 1.0940821650396037e-07,
 8.68326903788047e-05,
 10476.845089935949,
 3.409627939858213e-06,
 5940.568524468884,
 3311.413713368865,
 1.1978452140607235e-05,
 9570.917305950901,
 2.0605566052164028e-07,
 2.0605547136859186e-07,
 4771.275410076302,
 9070.360896331335,
 5094.578597602159,
 0.0008681703714453759,
 298.05541412797754,
 1.3733785779928225e-07,
 1.373377740884933e-07,
 8752.157731808133,
 1.8452547263016092e-07,
 7387.071449128779,
 386.31491479153516,
 301.45846126882657,
 4.5982519744585025e-08,
 4.598251035234828e-08,
 15125.1